# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

In [ ]:
subject_id = 20902040

In [ ]:
diff_data_df = pd.read_pickle('lib/diff-data.pkl')

In [ ]:
def get_losses_by_user(subject_id):
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = 1 - np.array(diff_data['mask'])[::-1]
  galaxy_data = np.array(diff_data['imageData'])[::-1]
  image_size = diff_data['width']
  size_diff = image_size / diff_data['imageWidth']
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(parsing.parse_annotation, size_diff=size_diff)

  tqdm.pandas(desc='[{}] Rendering models'.format(subject_id), leave=False)
  rendered = models.progress_apply(rendering.calculate_model, image_size=image_size, psf=psf).rename('rendered')
  tqdm.pandas(desc='[{}] Calculating differences'.format(subject_id), leave=False)
  losses = rendered.apply(fitting.loss, args=(galaxy_data, pixel_mask)).rename('loss')
  pd.Series(losses.values, index=names.values)
  all_losses = pd.Series(losses.values, index=names.values).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(~all_losses.duplicated(keep=False)).dropna(), # all non-duplicates
    all_losses.where(all_losses.duplicated(keep=False)).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [ ]:
losses = get_losses_by_user(subject_id)
losses

In [ ]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())